# Pandas 1

## Samuel Goldrup

## Math 403

## 30 August 2023

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

# Problem 1

In [2]:
# Prob 1
def prob1(file='budget.csv'):
    """"
    Read in budget.csv as a DataFrame with the index as column 0 and perform each of these operations on the DataFrame in order. 
    
    1) Reindex the columns such that amount spent on groceries is the first column and all other columns maintain the same ordering.
    2) Sort the DataFrame in descending order based on how much money was spent on Groceries.
    3) Reset all values in the 'Rent' column to 800.0.
    4) Reset all values in the first 5 data points to 0.0
    
    Return the values of the updated DataFrame as a NumPy array.
    
    Parameters:
        file (str): name of datafile
        
    Return:
        values (ndarray): values of DataFrame
    """
    df = pd.read_csv(file,index_col=0)
    
    cols = list(df.columns)
    cols.remove('Groceries')
    df = df.reindex(columns=['Groceries']+cols) #make groceries the first column
    
    df = df.sort_values('Groceries',ascending=False) #sort the values in the groceries column in descending order
    df['Rent'] = 800.0
    df.iloc[:5] = 0.0
    return df.to_numpy()

In [3]:
prob1()

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [174., 800.,  90.,  37.,  30.,  23.,   8.],
       [174., 800.,  82.,  35.,  nan,  26.,  nan],
       [172., 800.,  82.,  31.,  30.,  26.,   8.],
       [171., 800.,  82.,  40.,  nan,  23.,  nan],
       [171., 800.,  82.,  35.,  nan,  27.,  nan],
       [171., 800.,  80.,  30.,  31.,  22.,  nan],
       [170., 800.,  90.,  34.,  33.,  nan,   8.],
       [170., 800.,  85.,  34.,  nan,  25.,  nan],
       [167., 800.,  92.,  30.,  nan,  29.,  nan],
       [163., 800.,  85.,  30.,  nan,  nan,  nan],
       [163., 800.,  90.,  31.,  nan,  25.,  nan],
       [161., 800.,  85.,  30.,  nan,  24.,  nan],
       [160., 800.,  91.,  32.,  28.,  23.,  nan],
       [158., 800.,  92.,  nan,  nan,  22.,  nan],
       [157., 800.,  82.,  nan,

# Problem 2

In [4]:
# Prob 2
def prob2(file='budget.csv'):
    """
    Read in file as DataFrame.
    Fill all NaN values with 0.0.
    Create two new columns, 'Living Expenses' and 'Other'. 
    Sum the columns 'Rent', 'Groceries', 'Gas' and 'Utilities' and set it as the value of 'Living Expenses'.
    Sum the columns 'Dining Out', 'Out With Friends' and 'Netflix' and set as the value of 'Other'.
    Identify which column, other than 'Living Expenses' correlates most with 'Living Expenses'
    and which column other than 'Other' correlates most with 'Other'.

    Return the names of each of those columns as a tuple.
    The first should be of the column corresponding to \li{'Living Expenses'} and the second to \li{'Other'}.
    
    Parameters:
        file (str): name of datafile
        
    Return:
        values (tuple): (name of column that most relates to Living Expenses, name of column that most relates to Other)
    """
    df = pd.read_csv(file)
    df.fillna(0.0) #replace nans with zeros
    df['Living Expenses'] = df['Rent'] + df['Groceries'] + df['Gas'] + df['Utilities']
    df['Other'] = df['Dining Out'] + df['Out With Friends'] + df['Netflix']
    corr = df.corr() #initialize correlation dataframe object
    corr.loc['Other','Other'] = -2.0
    corr.loc['Living Expenses','Living Expenses'] = -2.0 #we don't care about correlation with itself
    corr['Living Expenses'].dropna() #remove nulls
    corr['Other'].dropna()
    return corr['Living Expenses'].dropna().idxmax(), corr['Other'].dropna().idxmax() 


In [5]:
prob2()

('Rent', 'Dining Out')

# Problem 3

In [6]:
def prob3(file='crime_data.csv'):
    """
    Read in crime data and use pandas to answer the following questions.
    
    Set the index as the column 'Year', and return the answers to each question as a tuple.
    
    1) Identify the three crimes that have a mean over 1,500,000. 
    Of these three crimes, which two are very correlated? 
    Which of these two crimes has a greater maximum value?
    Save the title of this column as a variable to return as the answer.
    
    2) Examine the data since 2000.
    Sort this data (in ascending order) according to number of murders.
    Find the years where Aggravated Assault is greater than 850,000.
    Save the indices (the years) of the masked and reordered DataFrame as a NumPy array to return as the answer.
    
    3) What year had the highest crime rate? 
    In this year, which crime was committed the most? 
    What percentage of the total crime that year was it? 
    Save this value as a float.
    
    
    Parameters:
        file (str): data
    
    Return:
        ans_1 (string): answer to Question 1
        ans_2 (ndarray): answer to Question 2
        ans_3 (float): answer to Question 3
    """
    #code to answer question 1
    df = pd.read_csv(file,index_col='Year')
    df_c = df.iloc[:,2:] #filer out unwanted rows,columns
    major_crimes = df_c.mean()[df_c.mean() > 1500000].index.tolist() #build mask
    corrs = df_c[major_crimes].corr() #mask through
    corrs[corrs == 1.0] = 0.0 #we don't care about corr with self
    max_val = 0.0
    max_pair = []
    for crime in major_crimes:
        if corrs.loc[crime].max() > max_val: #find the highest corr variable
            max_pair = [crime,corrs.loc[crime].idxmax()]
    ans_1 = df[max_pair].max().idxmax()
    
    #code to answer question 2
    df_modern = df.iloc[2000-df.index.tolist()[0]:,1:] #get years past 2000
    df_modern = df_modern['Murder'].sort_values()
    ans_2 = np.array(df_modern[df['Aggravated Assault'] > 850000].index.tolist())
    
    #code to answer question 3
    worst_year = (df['Total'] / df['Population']).idxmax() #get worst year based on rate
    worst_crime_wy = df.iloc[worst_year-df.index.tolist()[0],2:].idxmax() 
    
    ans_3 = 100*(df.loc[worst_year,worst_crime_wy] / df.loc[worst_year,'Total'])
    
    return ans_1,ans_2,ans_3

In [7]:
prob3()

('Property',
 array([2000, 2001, 2002, 2003, 2005, 2007, 2006]),
 89.97188308734142)

# Problem 4

In [8]:
def prob4(file='DJIA.csv'):
    """

    Read the data with a DatetimeIndex as the index.
    Drop rows any rows without numerical values, cast the "VALUE" column to floats, then return the updated DataFrame.

    Parameters:
        file (str): data file
    Returns:
        df (DataFrame): updated DataFrame of stock market data
    """
    df = pd.read_csv(file)
    df['DATE'] = pd.to_datetime(df['DATE'],format = "%Y-%m-%d") #convert to floats
    df.set_index('DATE',inplace=True) #run this function in place to save memwee
    df['VALUE'] = df['VALUE']._convert(numeric=True)
    df.dropna(inplace=True) #run this function inplace
    
    return df

In [9]:
prob4()

,VALUE
DATE,
2006-09-27,11689.24
2006-09-28,11718.45
2006-09-29,11679.07
2006-10-02,11670.35
2006-10-03,11727.34
...,...
2016-09-20,18129.96
2016-09-21,18293.70
2016-09-22,18392.46


# Problem 5

In [10]:
def prob5(file='paychecks.csv'):
    """

    Create data_range for index of paycheck data.

    Parameters:
        file (str): data file
    Returns:
        df (DataFrame): DataFrame of paycheck data
    """
    df = pd.read_csv(file,header=None)
    #periods 93, frequency is 14Days since payday is every two weeks
    df.set_index(pd.date_range(start='3/14/2008',periods=93,freq='14D'),inplace=True)
    
    return df

#column

In [11]:
prob5()

,0
2008-03-14,1122.26
2008-03-28,921.03
2008-04-11,962.46
2008-04-25,1035.97
2008-05-09,1078.59
...,...
2011-07-29,1095.53
2011-08-12,1018.39
2011-08-26,1027.08
2011-09-09,1005.90


# Problem 6

In [12]:
def prob6(file='DJIA.csv'):
    """
    Compute the following information about the DJIA dataset
    1. The single day with the largest gain
    2. The single day with the largest loss

    Parameters:
        file (str): data file
    Returns:
        max_day (<M8[ns]): DateTimeIndex of maximum change
        min_day (<M8[ns]): DateTimeIndex of minimum change
    """
    df = prob4(file) #run problem 4 function
    delta = df - df.shift(1) #take differences
    return delta.idxmax(), delta.idxmin()

In [13]:
prob6()

(VALUE   2008-10-13
 dtype: datetime64[ns],
 VALUE   2008-09-29
 dtype: datetime64[ns])